Creating a Unigram Tokenizer on the new sentences (without considering the definitions)
----------------------------------
We extracted new sentences from the book <i style="color: cyan">Grammaire de Wolof Moderne</i> by Pathé Diagne. Since we want to test the relevancy of the new sentences to our translation task, let us create a tokenizer for them. 

The process is almost the same as in [processing_4](text_processing4.ipynb) excepted that we will create another custom dataset for the custom transformer model.

Let us import the necessary libraries.

In [1]:
# for creating the tokenizer
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    normalizers
)

# for importing and manipulating the sentences
import pandas as pd
import random

# for plotting the box plot of the sequence lengths
import plotly.express as px

# for loading sentences with the custom dataset
from torch.utils.data import DataLoader

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load dataset and create generator

We will create one tokenizer for both of the French and Wolof corpora because the `T5` model' understand only one embedding layer. So we must create one generator for both of the French and Wolof corpora. 

In [2]:
# load sentences
sentences = pd.read_csv("data/extractions/new_data/sentences.csv")

# initialize a batch size
BATCH_SIZE = 400

# create generators (for the corpora)
def generate_sents():
    
    # recuperate the sentences
    french_sents = sentences['french'].to_list() 
    
    wolof_sents = sentences['wolof'].to_list() 
    
    sents = french_sents + wolof_sents
    
    for i in range(1, len(sents), BATCH_SIZE):
        
        yield sents[i:i+BATCH_SIZE]

#### Initialize the tokenizer

In [3]:
tokenizer = Tokenizer(models.Unigram())

#### Add normalizer

In [4]:
tokenizer.normalizer = normalizers.Replace(" {2,}", " ")

#### Configure the pre-tokenizers

We will use the Metaspace pre-tokenizer which separates the words considering the spaces between them. It will replace the space by a character (by default the underscore "_").

In [5]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

#### Initialize the trainers

In [6]:
special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>']

In [7]:
trainer = trainers.UnigramTrainer(special_tokens=special_tokens, unk_token = "<unk>") # let us take the default vocab size

#### Train the tokenizer

In [8]:
tokenizer.train_from_iterator(generate_sents(), trainer)

Let us print the vocab size.

In [9]:
print(f"Number of tokens: {tokenizer.get_vocab_size()}")

Number of tokens: 1935


#### Initialize the decoder

In [10]:
tokenizer.decoder = decoders.Metaspace()

#### Save the tokenizer

In [11]:
tokenizer.save("wolof-translate/wolof_translate/tokenizers/bart_tokenizers/tokenizer_v3.json")

#### Make a little example

Let us recuperate random sentences from the corpora and tokenize them.

In [12]:
random.seed(200)

french_sentence = '<s>' + random.choice(sentences['french']) + "</s>"

wolof_sentence = '<s>' + random.choice(sentences['wolof']) + "</s>"

In [13]:
# print the french sentence
french_sentence

'<s>Fais sortir tout cheval que tu vois !</s>'

In [14]:
# print the wolof sentence
wolof_sentence

'<s>Yeena doon dem</s>'

In [15]:
french_encoding = tokenizer.encode(french_sentence)

print("French tokens")
print(french_encoding.tokens)

print("French ids")
print(french_encoding.ids)

French tokens
['<s>', '▁Fa', 'is', '▁sortir', '▁tout', '▁cheval', '▁que', '▁tu', '▁vois', '▁!', '</s>']
French ids
[0, 162, 249, 535, 132, 392, 25, 42, 218, 16, 2]


In [16]:
wolof_encoding = tokenizer.encode(wolof_sentence)

print("Wolof tokens")
print(wolof_encoding.tokens)

print("Wolof ids")
print(wolof_encoding.ids)

Wolof tokens
['<s>', '▁Yee', 'na', '▁doon', '▁dem', '</s>']
Wolof ids
[0, 858, 208, 203, 14, 2]


#### Creating the custom dataset for the new sentences

We have two possibilities to use the tokenizer for fine-tuning a Bart model. 

- We can use the `PreTrainedTokenizerFast` class for which we will provide the different special tokens.

In [17]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer1 = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

- Or give directly the tokenizer to the `BartTokenizerFast` class.

In [18]:
from transformers import BartTokenizerFast

wrapped_tokenizer2 = BartTokenizerFast(
    tokenizer_object=tokenizer
)

Let us give them the sentences that we use as example. 

In [19]:
fr_encoding = wrapped_tokenizer1(french_sentence, max_length=15, padding='max_length', truncation=True)

fr_encoding

{'input_ids': [1, 1, 1, 1, 0, 162, 249, 535, 132, 392, 25, 42, 218, 16, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
wf_encoding = wrapped_tokenizer2(wolof_sentence, max_length=15, padding='max_length', truncation=True)

wf_encoding

{'input_ids': [0, 1931, 477, 20, 203, 14, 2, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}

Let us decode the wolof sentence.

In [21]:
wrapped_tokenizer1.decode(wf_encoding.input_ids, skip_special_tokens=True)

'Yeena doon dem'

Let us verify, before creating the custom dataset, the max length that we can get from the corpora' tokens without considering the augmentation. We must for that trace the box plot of the lengths and identify the range in which we will sample the max length of the sequences.

In [22]:
length = []

for sent in sentences['french'].to_list() + sentences['wolof'].to_list():
    
    len_ids = len(wrapped_tokenizer2(sent).input_ids)
    
    length.append(len_ids)

        

In [23]:
fig = px.box(x = length)

fig.update_layout({'xaxis': {'title': 'length'}})

The upper fence is of **16** and the max length is equal to **44**. Then we will test any value between the two. 

But considering the augmentation we can obtain more than the value that we will take because it will add modifications on the words and then it can recognize only parts of them and divide them in multiple other tokens. We will add to the max length the fifth of it. 